In [1]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
# from torch.nn import LogSoftmax
from torch import flatten

In [2]:
class LeNet(Module):
    def __init__(self, numChannels = 2):
        # call the parent constructor
        super(LeNet, self).__init__()

        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels=numChannels,\
                            out_channels=4, kernel_size=(2, 6))#2 - height, 6 - width 
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(3, 3), stride=(3, 3)) #stride = step
        
        
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=4, out_channels=8,\
                            kernel_size=(4, 4))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(3, 3), stride=(3, 3))
        
        
        # initialize third set of CONV => RELU => POOL layers
        self.conv3 = Conv2d(in_channels=8, out_channels=16,\
                            kernel_size=(3, 3))
        self.relu3 = ReLU()
        self.maxpool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        
        
        
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=32, out_features=8)
        self.relu4 = ReLU()
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=8, out_features=1)
#         self.logSoftmax = LogSoftmax(dim=1)
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
#         print(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)


        # flatten the output from the previous layer and pass it
        # through our only set of FC => RELU layers

        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu4(x)

        # pass the output to our softmax classifier to get our output
        # predictions
        output = self.fc2(x)
    #     output = self.logSoftmax(x)
        # return the output predictions
        return output

In [3]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# from sklearn.metrics import classification_report

import glob
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torchvision.transforms import ToTensor

from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import argparse
import torch
import time

In [4]:
# define training hyperparameters
INIT_LR = 1e-3
EPOCHS = 10

# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

        target_954 = np.load('data_segment/Y_natur_954.npz')
        for j in range(3000):
            self.data.append(["data_segment/natur_X/954_" + str(j) + '.npz',target_954['arr_0'][j]])
#         print(self.data)

        target_9_7 = np.load('data_segment/Y_natur_9_7.npz')
        for j in range(3000):
            self.data.append(["data_segment/natur_X/9_7_" + str(j) + '.npz',target_9_7['arr_0'][j]])

        target_Air = np.load('data_segment/Y_natur_Air.npz')
        for j in range(3000):
            self.data.append(["data_segment/natur_X/Air_" + str(j) + '.npz',target_Air['arr_0'][j]])
        
        temp = np.load('data_segment/natur_X/954_0.npz')
        self.img_dim = temp['arr_0'].shape  
        print(self.img_dim)
        
        
    def __len__(self):
        return len(self.data)    
    
    
    def __getitem__(self, idx):
        img_path, target = self.data[idx]
        
        img = np.load(img_path)
        img_tensor = torch.from_numpy(img['arr_0'])
        
        target_tensor = torch.tensor(target)
#         print(target_tensor, target)
        
#         img_tensor = img_tensor.permute(2, 0, 1)
#         class_id = torch.tensor([class_id])
        
        return img_tensor, target_tensor

In [6]:
data = CustomDataset()

print(len(data))

(2, 64, 50)
9000


In [7]:
print(data[1][0].size())
print(data[1][0].type())
print(data[1][1].type())
print(data[1][1], data[1][1].item())


torch.Size([2, 64, 50])
torch.DoubleTensor
torch.DoubleTensor
tensor(0.0319, dtype=torch.float64) 0.03186977247219236


In [8]:
Train_split = 0.7
Val_split = 0.15
Test_split = 0.15

(trainData, valData, testData) = random_split(data, [Train_split, Val_split, Test_split],\
                                              generator=torch.Generator().manual_seed(42))

In [9]:
print(len(trainData))
print(len(valData))
print(len(testData))

6300
1350
1350


In [14]:
BATCH_SIZE = 64
trainDataLoader = DataLoader(trainData, shuffle=True, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)



# print(testDataLoader.dataset[0])

# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
print(trainSteps)

valSteps = len(valDataLoader.dataset) // BATCH_SIZE
print(valSteps)


# print(trainDataLoader.dataset)
# print(trainDataLoader.dataset[1])
# print(trainDataLoader.dataset[1][0].type())
# print(trainDataLoader.dataset[1][1])

98
21


In [15]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")

model = LeNet(numChannels = 2).to(device)
print(model)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossMSE = nn.MSELoss()
# initialize a dictionary to store training history
H = {"train_loss": [],"val_loss": []}
# measure how long training is going to take
print("[INFO] training the network...")

# loss = nn.MSELoss()
# print(nn.MSELoss(torch.zeros(64, requires_grad=True),torch.ones(64)))

[INFO] initializing the LeNet model...
LeNet(
  (conv1): Conv2d(2, 4, kernel_size=(2, 6), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(4, 4), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32, out_features=8, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)
[INFO] training the network...


In [16]:
startTime = time.time()


# loop over our epochs
for e in range(0, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
   
    # loop over the training set
    for (x, y) in trainDataLoader:
#         print(y)
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
        print(y.size(), y.type())
        print(x.size(), x.type())
#         print(x)
        # perform a forward pass and calculate the training loss
        pred = model(x)
#         print(pred)

        loss = lossMSE(pred, y)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        totalTrainLoss += loss
    # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        for (x, y) in valDataLoader:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))
            # make the predictions and calculate the validation loss
            pred = model(x)
            totalValLoss += lossMSE(pred, y)

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Val loss: {:.6f}".format(
        avgTrainLoss, avgValLoss))
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: \
        {:.2f}s".format(endTime - startTime))

torch.Size([64]) torch.cuda.DoubleTensor
torch.Size([64, 2, 64, 50]) torch.cuda.DoubleTensor


RuntimeError: Input type (double) and bias type (float) should be the same

In [17]:
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
    # set the model in evaluation mode
    model.eval()

    # initialize a list to store our predictions
    preds = []
    # loop over the test set
    for (x, y) in testDataLoader:
        # send the input to the device
        x = x.to(device)
        # make the predictions and add them to the list
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())


[INFO] evaluating network...


RuntimeError: Input type (double) and bias type (float) should be the same

In [ ]:
# loss = nn.MSELoss()
# print(loss(torch.tensor(np.array([1,2])),torch.tensor(np.array([1,0]))))